In [119]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd



In [87]:
# paramter set up 
s0=100
r=0.05
vol=0.10
T=2
noofsteps=100
noofpaths=10000

def monte_carlo(s0,r,T,vol,noofsteps,noofpaths):
    dt=T/noofsteps
    z=np.random.standard_normal((noofpaths,noofsteps))
    rets = (r - 0.5*vol**2)*dt + vol*z*np.sqrt(dt)
    cum_rets = np.cumsum(rets,axis=1)
    paths = np.exp(cum_rets)*s0
    paths = np.hstack([s0*np.ones((noofpaths,1)),paths])
    time = np.arange(0,noofsteps+1)*dt
    return paths,time
    
def American_option_pricer(s0,strike,vol,r,T,noofsteps,noofpaths,type):
    paths,time =monte_carlo(s0,r,T,vol,noofsteps,noofpaths)
    type = type.lower()
    dt= T/noofsteps
    cashflows = np.zeros((noofpaths,noofsteps+1))
    discount = np.exp(-r*dt)
    Excercise_payoff_immediate = np.maximum(paths-strike,0) if type=='call' else np.maximum(strike-paths,0)
    cashflows[:,-1] = Excercise_payoff_immediate[:,-1]
    for t in range(noofsteps-1,0,-1):
                itm = Excercise_payoff_immediate[:,t]>0
                x=paths[itm,t]
                y= cashflows[itm,t+1]*discount
                if len(x)==0:
                    continue
                coeffs = np.polyfit(x,y,2)
                continuation= np.polyval(coeffs,x)
                excercise = Excercise_payoff_immediate[itm,t] > continuation
                cashflows[itm,t] = np.where(excercise,Excercise_payoff_immediate[itm,t],cashflows[itm,t+1]*discount)
                not_itm = ~itm
                cashflows[not_itm,t] = cashflows[not_itm,t]*discount
                price =  np.mean(cashflows[:,1]*discount)
    return price
    
paths,time=monte_carlo(s0,r,T,vol,noofsteps,noofpaths)
American_option_pricer(s0,100,vol,r,T,noofsteps,noofpaths,'put')


np.float64(0.6870374800343878)